In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot
# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt'))
print(fn_txt[:3])
fn_cycles = Path('tbmData/cycles1')

debug = True

mulr = 3
# num_cycles = 3481
num_cycles = 10
valid_ratio = 0.2
train_ratio = 1 - valid_ratio
train_idx = np.arange(int(num_cycles * valid_ratio), num_cycles)
valid_idx = np.arange(int(num_cycles * valid_ratio))
train_idx_tile = (train_idx[:, None] + np.arange(mulr) * num_cycles).flatten()
valid_idx_tile = (valid_idx[:, None] + np.arange(mulr) * num_cycles
                  ).flatten()  # take from all tiles

bs = 2 if debug else int(num_cycles * train_ratio)
gpu_start = 1
torch.cuda.set_device(gpu_start)
device_ids = range(gpu_start, gpu_start + num_gpus)

# cont_names = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速','主液压油箱温度', '前点偏差X', '主液压油箱温度']
is_problem1 = True
dep_var = ['推进速度电位器设定值', '刀盘转速电位器设定值'] if is_problem1 else ['总推进力', '刀盘扭矩']

In [ ]:
# with concurrent.futures.ThreadPoolExecutor() as e:
#     cycles = tqdm_notebook(e.map(read_feather_fn, sorted(fn_cycles.glob('cycle*'))[:num_cycles]), desc='read_dataframe', total=num_cycles)
#     idx = list(tqdm_notebook(e.map(beginning_index, [o.iloc[:500] for o in cycles]), desc='beginning', total=num_cycles))
# #     cont = [o.loc[:,cont_names].iloc[i:i+30] for i, o in zip(idx, cycles)]
#     cont = [o.iloc[:,3:].iloc[i:i+30] for i, o in zip(idx, cycles)]

In [ ]:
fns_feathers = sorted(fn_cycles.glob('cycle*'))[:num_cycles]
fmtr = DataFormatter(cycle_feathers=fns_feathers)
cycles = fmtr.cycles
idx = [beginning_index(o.iloc[:500]) for o in tqdm_notebook(cycles)]

In [ ]:
cont_names = [o for o in cycles[0].columns[2:] if o not in dep_var]
df_conts = tile_with_noise(cycles, idx, mulr, cont_names)

In [ ]:
# cyc = cycles[0].iloc[:500]
# axis = plots(get_interesting_columns(cyc), title=str(idx));
# for ax in axis.flatten():
#     ax.axvline(idx)

In [ ]:
deps = fmtr.get_y(dep_var)
deps = pd.concat([deps]*mulr).reset_index(drop=True)
cyc_cont = flatten_and_cat(df_conts, deps)

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all_5c')

In [ ]:
#init
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all_5c')

In [ ]:
from fastai.tabular.data import OptTabTfms
class MultiDeptTabularDataBunch(DataBunch):
    "Create a `DataBunch` suitable for tabular data."

    @classmethod
    def from_df(cls, path, df:DataFrame, dep_var:str, valid_idx:Collection[int], procs:OptTabTfms=None,
                cat_names:OptStrList=None, cont_names:OptStrList=None, classes:Collection=None, 
                test_df=None, **kwargs)->DataBunch:
        "Create a `DataBunch` from `df` and `valid_idx` with `dep_var`."
        cat_names = ifnone(cat_names, []).copy()
        cont_names = ifnone(cont_names, list(set(df)-set(cat_names)-set(dep_var)))
        procs = listify(procs)
        src = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(valid_idx))
        src = src.label_from_df(cols=dep_var) if classes is None else src.label_from_df(cols=dep_var, classes=classes)
        if test_df is not None: src.add_test(TabularList.from_df(test_df, cat_names=cat_names, cont_names=cont_names,
                                                                 processor = src.train.x.processor))
        return src.databunch(**kwargs)

In [ ]:
# stacked = pd.concat([cyc_cont.iloc[int(valid_ratio * num_cycles):]]*mulr)
stacked = cyc_cont
print(stacked.shape)
valid_idx_tile = (valid_idx[:,None] + np.arange(mulr) * num_cycles).flatten() # take from all tiles
procs = Normalize
databunch = MultiDeptTabularDataBunch.from_df('tmp', stacked, dep_var, valid_idx=valid_idx, bs=bs, procs=procs)

In [ ]:
from fastai.text import *

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')

In [ ]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv', bs=bs, bptt=3)
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
def ni(it): return next(iter(it))
x, y = ni(data_lm.valid_dl)

In [ ]:
y[0].shape

In [ ]:
t, v = databunch.train_ds, databunch.valid_ds

In [ ]:
t.x.items = t.x.xtra.values.T

In [ ]:
lm = TextLMDataBunch.create(t, v, no_check=False, bs=bs, bptt=30)

In [ ]:
x, y = lm.train_ds[0]

In [ ]:
x.data[1].shape

In [ ]:
lm.train_dl.dl.collate_fn(lm.train_dl.dl.dataset[1])

In [ ]:
lm.train_dl.

In [ ]:
vars(lm.train_dl.dl.dataset)

In [ ]:
class RNNDataBunch(DataBunch):
    "Create a `TextDataBunch` suitable for training a language model."
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', no_check:bool=False, bs=64, num_workers:int=0,
               device:torch.device=None, collate_fn:Callable=data_collate, dl_tfms:Optional[Collection[Callable]]=None, 
               **kwargs) -> DataBunch:
        "Create a `TextDataBunch` in `path` from the `datasets` for language modelling."
        datasets = cls._init_ds(train_ds, valid_ds, test_ds)
#         datasets = [LanguageModelPreLoader(ds, shuffle=(i==0), bs=bs, **kwargs) for i,ds in enumerate(datasets)]
        val_bs = bs
        dls = [DataLoader(d, b, shuffle=False) for d,b in zip(datasets, (bs,val_bs,val_bs,val_bs)) if d is not None]
        return cls(*dls, path=path, device=device, dl_tfms=dl_tfms, collate_fn=collate_fn, no_check=no_check)

In [ ]:
databunch.train_d

In [ ]:
rnndb = RNNDataBunch.create(t, v, bs=2)

In [ ]:
x, y = ni(rnndb.train_dl)

In [ ]:
x[1].shape

In [ ]:
learner = tabular_learner(databunch, [5000, 5000, 1000])

from fastai.callbacks.tracker import *
learner.callback_fns += [ShowGraph, SaveModelCallback]
# learner.model = torch.nn.DataParallel(learner.model, device_ids=device_ids)

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(1, 1e-1)

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(100, 1e-2)

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(3, 1e-2/2)
learner.recorder.plot_lr()

In [ ]:
learner.fit_one_cycle(10, 1e-3/2)
learner.recorder.plot_lr()

In [ ]:
x, y = next(iter(databunch.valid_dl))
y = to_np(y)
pred = to_np(learner.pred_batch(x)).squeeze()

In [ ]:
r = y.max() - y.min()
list(zip(pred, y, [(a-b)/(r)*100 for a,b in zip(pred, y)]))

In [ ]:
learner.fit_one_cycle(50, 1e-3)

In [ ]:
learner.fit_one_cycle(100, 2*1e-3)